In [2]:
# === Header: Setup SQLite and Connect to DB ===
import sqlite3
import pandas as pd

# Connect to your SQLite database
conn = sqlite3.connect('pokemon_data.db')  # or whatever your DB file is named
cursor = conn.cursor()

# Helper function to run SQL queries and show results as DataFrame
def run_query(query):
    return pd.read_sql_query(query, conn)

# Example usage (you can delete this later)
run_query("""
    SELECT * FROM pokemon
    LIMIT 5
    """)


,id,name,generation,type1,type2,hp,attack,defense,special_attack,special_defense,speed,bst,stage,is_fully_evolved,dex_entry
0,1,bulbasaur,1,grass,poison,45,49,49,65,65,45,318,1,0,A strange seed was planted on its back at birt...
1,2,ivysaur,1,grass,poison,60,62,63,80,80,60,405,2,0,"When the bulb on its back grows large, it appe..."
2,3,venusaur,1,grass,poison,80,82,83,100,100,80,525,3,1,The plant blooms when it is absorbing solar en...
3,4,charmander,1,fire,None,39,52,43,60,50,65,309,1,0,"Obviously prefers hot places. When it rains, s..."
4,5,charmeleon,1,fire,None,58,64,58,80,65,80,405,2,0,"When it swings its burning tail, it elevates t..."


# Analysis of Pokémon Data

This project investigates the factors contributing to a Pokémon's perceived "strength" by examining its statistics, types, and generational origin. Pokémon is a widely popular game featuring creatures known as Pokémon, which engage in combat utilizing various elemental types such as Fire, Water, and Electric. Each Pokémon is defined by six primary statistics:

- **HP** (Hit Points) – durability against damage
- **Attack** – offensive capability with physical attacks
- **Defense** – defensive capability against physical attacks
- **Sp. Atk** – offensive capability with special attacks
- **Sp. Def** – defensive capability against special attacks
- **Speed** – determines action order in combat
    The sum of these six statistics constitutes a Pokémon’s **Base Stat Total (BST)**, which is commonly employed as a metric for assessing overall capability.
    There are currently **9 generations** of Pokémon, each introducing new creatures, types, and mechanics. This analysis aims to address questions such as:
- Which Pokémon exhibit the highest statistical potential?
- Which type exhibits the most effective offensive capability (inflicts the highest amount of super-effective damage)?
- How do different types and generations compare in terms of potential strength?
    Visualizations and statistical methods are utilized to uncover patterns and insights across a comprehensive dataset of Pokémon.

## Identifying the Strongest Pokémon

Determining the "strongest" Pokémon can be approached using various criteria.

1. Pokémon with the highest Base Stat Total (BST) could be considered strongest, in which case the following Pokémon are candidates:

In [3]:
run_query("""
    SELECT id, name, bst
    FROM pokemon
    ORDER by bst desc
    LIMIT 5
    """)

,id,name,bst
0,493,arceus,720
1,890,eternatus,690
2,150,mewtwo,680
3,249,lugia,680
4,250,ho-oh,680


Utilizing BST as the criterion, Arceus emerges as the leading contender. However, all five of these Pokémon are classified as Legendary, typically encountered only within the narrative context of the game's storyline due to their significance to the lore. Their exceptional strength is often attributed to this narrative positioning.
Regrettably, the available data lacks a specific attribute for filtering based on Legendary status. However, filtering by evolution stage is feasible. Numerous Pokémon undergo evolution throughout the game's progression; conversely, most Legendary Pokémon do not evolve. Consequently, examining Pokémon with the highest BST that possess more than one evolution stage may yield more relevant insights:

In [4]:
run_query("""
    SELECT id, name, bst
    FROM pokemon
    WHERE stage != 1
    ORDER by bst desc
    LIMIT 10
    """)

,id,name,bst
0,791,solgaleo,680
1,792,lunala,680
2,289,slaking,670
3,149,dragonite,600
4,248,tyranitar,600
5,373,salamence,600
6,376,metagross,600
7,445,garchomp,600
8,490,manaphy,600
9,635,hydreigon,600


As observed, this table largely excludes Legendary Pokémon. Only Solgaleo, Lunala, and Manaphy remain, as they are exceptional cases possessing evolution stages. This represents the closest feasible method within the data to approximate the exclusion of Legendary Pokémon.

2. Alternatively, a Pokémon's 'strength' could be assessed based on its offensive power. Effective offense often depends on either initiating action first (due to high Speed) or enduring the opponent's attack. For analytical simplicity, this examination will prioritize the scenario where the Pokémon acts first, as this may mitigate or negate incoming damage. The average Speed statistic for fully evolved Pokémon is calculated as follows:

In [5]:
run_query("""
    SELECT avg(speed) as avg_speed
    FROM pokemon
    WHERE is_fully_evolved = 1
    """)

,avg_speed
0,78.136194


Subsequently, Pokémon possessing the highest Attack or Special Attack statistics and a Speed value equal to or exceeding this average are identified:

In [6]:
run_query("""
    SELECT id, name, attack, speed
    FROM pokemon
    WHERE is_fully_evolved = 1 AND speed > (select avg(speed) from pokemon)
    ORDER by attack desc
    LIMIT 10
    """)

,id,name,attack,speed
0,798,kartana,181,109
1,289,slaking,160,100
2,486,regigigas,160,100
3,383,groudon,150,90
4,384,rayquaza,150,95
5,644,zekrom,150,90
6,612,haxorus,147,97
7,998,baxcalibur,145,87
8,567,archeops,140,110
9,794,buzzwole,139,79


In [7]:
run_query("""
    SELECT id, name, special_attack, speed
    FROM pokemon
    WHERE is_fully_evolved = 1 AND speed > (select avg(speed) from pokemon)
    ORDER by special_attack desc
    LIMIT 10
    """)

,id,name,special_attack,speed
0,796,xurkitree,173,83
1,150,mewtwo,154,130
2,806,blacephalon,151,107
3,382,kyogre,150,90
4,384,rayquaza,150,95
5,483,dialga,150,90
6,484,palkia,150,100
7,643,reshiram,150,90
8,720,hoopa,150,70
9,609,chandelure,145,80


These Pokémon could also be regarded as exceptionally capable, given their superior speed relative to the average and their potent offensive statistics.

## Analyzing Pokémon by Type
Pokémon are assigned elemental types, frequently possessing two types concurrently, referred to as a type combination. Type assignments dictate resistances and vulnerabilities; for instance, Fire-type Pokémon are susceptible to increased damage from Water-type attacks, whereas Grass-type Pokémon exhibit resistance to them. Type effectiveness modifiers also interact. Consequently, a Pokémon possessing both Grass and Fire typing would receive standard damage from Water-type attacks. A Type weakness results in a damage multiplier of two times the standard amount, while resistance reduces damage to half. Mutual weakness across both types yields a quadruple damage multiplier, whereas mutual resistance results in one-fourth the standard damage.

Certain Pokémon types are considered to possess superior offensive capabilities compared to others. The Ice type stands out as a prime example, possessing significant offensive strength against Ground, Flying, and Dragon types. A key characteristic of these three types is their limited number of weaknesses; each possesses only three. Notably, the Ice type represents a unique intersection, being the sole shared weakness among Ground, Flying, and Dragon. However, beyond these specific type interactions, the data presented below clearly illustrates the substantial number of Pokémon across the board that are susceptible to Ice-type attacks:

In [8]:
run_query("""
    WITH AllAttackingTypes AS (
    SELECT
        id AS attacking_type_id,
        name AS attacking_type_name -- Assuming type.name is consistently cased (e.g., "Grass", "Fire")
    FROM type
),
PokemonEffectivenessAcrossAllAttacks AS (
    SELECT
        p.id AS pokemon_id,
        att.attacking_type_id,
        att.attacking_type_name,
        CASE
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.no_damage_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.weak_to) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 2.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t1.resist_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.5
            ELSE 1.0
        END AS type1_multiplier,
        CASE
            WHEN p.type2 IS NULL THEN 1.0
            WHEN t2.name IS NULL THEN 1.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.no_damage_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.weak_to) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 2.0
            WHEN EXISTS (
                SELECT 1
                FROM json_each(t2.resist_from) j
                WHERE CAST(j.value AS INTEGER) = att.attacking_type_id
            ) THEN 0.5
            ELSE 1.0
        END AS type2_multiplier
    FROM
        pokemon p
    -- MODIFIED JOIN CONDITIONS:
    JOIN type t1 ON LOWER(p.type1) = LOWER(t1.name)       -- Case-insensitive join for Type 1
    LEFT JOIN type t2 ON LOWER(p.type2) = LOWER(t2.name)  -- Case-insensitive join for Type 2
    CROSS JOIN AllAttackingTypes att
),
PokemonTotalEffectivenessAcrossAllAttacks AS (
    SELECT
        pokemon_id,
        attacking_type_id,
        attacking_type_name,
        type1_multiplier * type2_multiplier AS total_multiplier
    FROM PokemonEffectivenessAcrossAllAttacks
)
SELECT
    att_summary.attacking_type_name AS "Attacking Type",
    SUM(CASE WHEN att_summary.total_multiplier = 2.0  THEN 1 ELSE 0 END) AS "Pokemon 2x Weak To",
    SUM(CASE WHEN att_summary.total_multiplier = 4.0  THEN 1 ELSE 0 END) AS "Pokemon 4x Weak To",
    SUM(CASE WHEN att_summary.total_multiplier = 0.5  THEN 1 ELSE 0 END) AS "Pokemon Resist (0.5x)",
    SUM(CASE WHEN att_summary.total_multiplier = 0.25 THEN 1 ELSE 0 END) AS "Pokemon 0.25x Resist",
    SUM(CASE WHEN att_summary.total_multiplier = 0.0  THEN 1 ELSE 0 END) AS "Pokemon Immune To"
FROM PokemonTotalEffectivenessAcrossAllAttacks att_summary
GROUP BY
    att_summary.attacking_type_id, att_summary.attacking_type_name
ORDER BY
    "Pokemon 2x Weak To" desc;
        """)

,Attacking Type,Pokemon 2x Weak To,Pokemon 4x Weak To,Pokemon Resist (0.5x),Pokemon 0.25x Resist,Pokemon Immune To
0,Ice,263,29,266,11,0
1,Ground,248,26,137,6,109
2,Rock,245,27,170,6,0
3,Fire,244,18,277,25,0
4,Fighting,241,17,230,52,65
5,Flying,240,13,158,12,0
6,Electric,208,8,210,9,75
7,Bug,170,14,314,63,0
8,Grass,168,29,346,102,0
9,Water,164,14,287,13,0


Based on the table presented above, it is evident that Ice-type moves deal double damage (2x) against 263 Pokémon. This figure represents the highest count in this category, placing Ice-type moves in the top position for 2x effectiveness. Furthermore, these moves inflict quadruple damage (4x) against 29 Pokémon, a count that ties with Grass-type moves for the highest number of Pokémon susceptible to 4x damage. Conversely, Ice-type moves are only half effective (0.5x damage) against 266 Pokémon, ranking sixth in terms of the number of Pokémon resistant to the type.

## Determining the Generally Best Pokémon Type
When considering the determination of the generally best Pokémon type, sole offensive strength, such as that exhibited by the Ice type, may not be sufficient criteria. Therefore, it is necessary to examine additional factors, including the average Base Stat Total (BST) of Pokémon belonging to each type.

In [9]:
run_query("""
    SELECT
        t.name AS type_name,
        ROUND(AVG(p.bst),2) AS avg_bst
    FROM
        pokemon p
    JOIN
        type t
    ON
        LOWER(p.type1) = LOWER(t.name) OR LOWER(p.type2) = LOWER(t.name)
    GROUP BY
        t.name
    ORDER BY
        avg_bst DESC
""")

,type_name,avg_bst
0,Dragon,504.76
1,Steel,479.72
2,Fighting,467.14
3,Dark,459.59
4,Psychic,457.45
5,Ice,453.65
6,Fire,449.51
7,Ghost,447.46
8,Electric,442.23
9,Rock,442.12


Based on the data presented above, the Dragon type appears to be the strongest based on average BST, boasting a mean BST of 504. This figure surpasses that of Steel, which ranks second, by a margin of 20 points. However, Dragon-type attacks are super effective against only 67 Pokémon, placing them in the seventeenth position in terms of super-effective targets, alongside the Normal type, which cannot deal super-effective damage to any type. Consequently, the strength of the Dragon type appears to be primarily attributed to its statistical profile rather than its offensive type effectiveness.

For those wondering why moves typing affect the process of ranking, the Same Type Attack Bonus (STAB) is a mechanic that grants a 1.5x damage multiplier when a Pokémon uses an attack that matches its own type. This bonus represents a significant factor in overall damage calculation.

The Steel type, occupying the second position in terms of average BST, while Steel-type attacks are super effective against approximately 140 Pokémon, is widely regarded as one of the premier defensive types. It possesses a notable 10 resistances (Normal, Grass, Ice, Flying, Psychic, Bug, Rock, Dragon, Fairy, and Steel), while being susceptible to only three types: Fire, Fighting, and Ground. Given its strong second-place ranking in average BST and its status as one of the most resilient defensive types, it can be argued that the Steel type is the generally best type overall, primarily due to its exceptional defensive profile and robust statistical average. The Dragon type follows closely in second place, distinguished by its impressive statistical base.

## Identifying the Generally Weakest Pokémon Type
In this instance, the average Base Stat Total (BST) ranking aligns with other indicators to identify the generally weakest type. The Bug type exhibits the lowest average BST at 374, standing as the sole type below the 400 threshold. Furthermore, Bug-type attacks demonstrate limited offensive efficacy: they deal reduced damage (0.5x) to a significant 7 types (Fire, Fighting, Poison, Flying, Ghost, Steel, and Fairy), while being super effective (2x) against only 3 types. The combination of its low average statistical profile and poor offensive type coverage strongly positions the Bug type as the weakest overall.

## Determining the Strongest/Weakest Pokémon Generation
Unlike some previous findings, the identification of the generation containing the strongest or weakest Pokémon can be readily determined by utilizing the average Base Stat Total (BST) metric. For the purpose of this analysis, Pokémon are assigned to the generation in which they first appeared. This methodology counts Pokémon debuting in a particular generation under that generation, even if they reappear in subsequent titles. While move pools may evolve and spread across generations, focusing on the average BST provides a consistent measure of the statistical baseline introduced by each generation.

In [10]:
run_query("""
    SELECT
        generation AS Generation,
        ROUND(AVG(bst), 2) AS Average_BST
    FROM pokemon
    WHERE
        is_fully_evolved = 1
        AND stage != 1
    GROUP BY generation
    ORDER BY Average_BST DESC
""")

,Generation,Average_BST
0,7,514.74
1,9,505.03
2,4,504.94
3,6,504.21
4,8,502.44
5,5,498.49
6,1,486.22
7,3,480.02
8,2,478.74


Based on the data, after excluding Legendary Pokémon and focusing exclusively on fully evolved species, Generation 7 is observed to have the highest average Base Stat Total (BST) at 514. This figure is notably higher than the second-ranked Generation 9, showing a difference of 9 points. In contrast, the generations ranking from second through sixth exhibit a much narrower range, with only 7 points separating their overall averages. This indicates that Generation 7's average represents a significant outlier compared to the others. Conversely, Generation 2 recorded the lowest overall average BST at 478, a result that may align with common discussions regarding Generation 2 as "the forgettable generation."

## What Type is the Fastest?
Determining the fastest type can be approached by calculating the average Speed for each Pokémon type.

In [11]:
run_query("""
    SELECT
        t.name AS type_name,
        ROUND(AVG(p.speed), 2) AS avg_speed
    FROM
        pokemon p
    JOIN
        type t
    ON
        LOWER(p.type1) = LOWER(t.name) OR LOWER(p.type2) = LOWER(t.name)
    WHERE
        p.is_fully_evolved = 1

    GROUP BY
        t.name
    ORDER BY
        avg_speed DESC
""")

,type_name,avg_speed
0,Electric,92.95
1,Flying,91.65
2,Dragon,88.65
3,Fighting,84.06
4,Fire,83.83
5,Dark,82.87
6,Poison,82.83
7,Psychic,81.11
8,Normal,80.68
9,Fairy,78.94


Based on the data, Electric types exhibit the highest average Speed. This finding is consistent with common understanding, as types frequently associated with high speed include Electric, Flying, Dragon, and Fighting. The average speeds among these faster types show a relatively narrow range; the fourth fastest type, Fighting, averages only 8 points less than the Electric type. Conversely, Rock types recorded the lowest average Speed, at 59, which also aligns with common perceptions of their typical mobility. This average for Rock types is notably lower than that of the fourth slowest type, Grass, showing a significant difference of 12 points, indicating that Rock types are the slowest on average by a considerable margin.

## What type is the bulkiest
To determine which type is the "bulkiest," we must consider several factors related to a Pokémon's defensive capabilities. Pokémon possess three primary defensive statistics: Hit Points (HP), Defense (DEF), and Special Defense (SPD).While calculating damage is a complex procedure involving multiple variables, a key distinction lies in the attack type. When a Pokémon is hit by a physical attack, its Defense stat is used in the calculation. Conversely, if the attacking move is special, the receiver's Special Defense stat is used instead.Hit Points, however, contribute equally to survivability regardless of the attack type. Given that a Pokémon must be able to withstand both physical and special attacks to be considered genuinely bulky, and for simplification in this analysis, a composite "Bulk" statistic will be calculated as follows:

$Bulk = HP+\frac{DEF+SPD}{2}$

This formula is employed because HP provides a universal defense, while the average of Defense and Special Defense provides a general measure of resilience against type-dependent attacks.

In [12]:
run_query("""
    SELECT
        t.name AS type_name,
        ROUND(AVG(p.hp + (p.defense + p.special_defense) / 2), 2) AS avg_bulk
    FROM
        pokemon p
    JOIN
        type t
    ON
        LOWER(p.type1) = LOWER(t.name) OR LOWER(p.type2) = LOWER(t.name)
    WHERE
        p.is_fully_evolved = 1
        AND p.stage != 1
    GROUP BY
        t.name
    ORDER BY
        avg_bulk DESC
""")

,type_name,avg_bulk
0,Steel,182.96
1,Rock,178.92
2,Ground,176.88
3,Dragon,176.45
4,Ice,173.53
5,Psychic,171.11
6,Fairy,167.16
7,Water,166.43
8,Normal,165.69
9,Ghost,162.27


Based on the data, the results maintain a degree of realism, identifying types such as Steel, Rock, and Ground as the bulkiest. Conversely, Bug types are found to be the least bulky, and this difference is evident by a noticeable margin.

# Moves
Pokémon moves are the specific actions a Pokémon can perform during battle. These actions are the fundamental means by which Pokémon interact, encompassing offensive attacks, status effects, and statistical modifications. As the primary tools of combat, moves are critically important to a Pokémon's strategic viability and determine its effectiveness in battle.

This table provides detailed attributes for each Pokémon move. The columns are defined as follows:

- **ID** A unique numerical identifier assigned to each distinct move entry in the dataset.
- **Name** – The official name of the Pokémon move, presented as a character string.
- **Accuracy** – The probability, typically expressed as a percentage or a value out of 100, that the move will successfully connect with the target during battle.
- **PP** – Power Points. Represents the maximum number of times a specific move can be utilized before it is depleted.
- **Priority** – A numerical value that determines the execution order of moves within a turn. Moves with higher priority values are generally - executed earlier than moves with lower or standard priority.
- **Power** – Base Power (BP). A numerical value indicating the inherent strength of an offensive move, serving as a primary factor in its damage - calculation. Moves with a power of 0 typically have non-damaging effects.
- **Damage_Class** – Categorizes the move based on how its damage is calculated or if it inflicts damage at all. Common values include 'Physical', - 'Special', and 'Status'.
- **Type** – The elemental or thematic typing assigned to the move (e.g., Fire, Water, Grass, Normal). This type interacts with the target's typing - to determine damage effectiveness.
- **Effect** – A descriptive text field detailing any secondary or additional effects the move may inflict, beyond its primary function (e.g., - inflicting a status condition, changing stats).
- **Effect_Chance** – The probability, typically expressed as a percentage, that the secondary effect described in the 'Effect' column will occur - upon successful execution of the move. Applies only to moves with secondary effects.
- **Target** – Specifies the intended recipient(s) or area of effect for the move (e.g., 'single target', 'all opponents', 'the user', 'all allies').

## Criteria for a Generally Good Move
To establish a baseline for evaluating moves, we begin by defining what constitutes a "generally good" move. Fundamentally, a move is considered good if it exhibits superiority or optimal characteristics compared to other moves of a similar kind.

In [15]:
run_query("""
    SELECT name as Move_Name, accuracy as Accuracy, pp as PP, power, type, effect
    FROM moves
    WHERE name = 'water-gun' or name = 'jet-punch'

""")

,Move_Name,Accuracy,PP,power,type,effect
0,jet-punch,100,15,60,water,No effect description available.
1,water-gun,100,25,40,water,Inflicts regular damage with no additional eff...


Based on the preceding data, an examination of moves like Jet Punch and Water Gun reveals that both are Water-type attacks lacking additional effects. The primary distinction lies in their Base Power and Power Points (PP). Jet Punch possesses a Base Power of 70, notably 20 points higher than Water Gun's 50. The only apparent disadvantage for Jet Punch is its PP count of 15, which is 10 less than Water Gun's 25. However, when determining the optimal attacking move, the difference between 15 and 25 PP is often considered less critical, as a single move is rarely utilized this many times within a typical battle.

In [22]:
run_query("""
    SELECT 
        type AS Move_Type,
        damage_class AS Damage_Class,
        name AS Move_Name,
        power AS Power,
        accuracy AS Accuracy,
        effect AS Effect
        
    FROM moves
    WHERE power = (
        SELECT MAX(power)
        FROM moves AS sub_moves
        WHERE sub_moves.type = moves.type AND sub_moves.damage_class = moves.damage_class
        AND pp > 1
        AND accuracy is NOT NULL
        
    )
    AND damage_class IN ('physical', 'special')
    ORDER BY Move_Type, Damage_Class
""")

,Move_Type,Damage_Class,Move_Name,Power,Accuracy,Effect
0,bug,physical,megahorn,120,85,Inflicts regular damage with no additional eff...
1,bug,special,bug-buzz,90,100,Has a 10% chance to lower the target's Special...
2,bug,special,pollen-puff,90,100,"Damages opponents, but heals allies for 50% of..."
3,dark,physical,foul-play,95,100,Calculates damage with the target's attacking ...
4,dark,special,fiery-wrath,90,100,Inflicts regular damage with no additional eff...
5,dragon,physical,outrage,120,100,"Hits every turn for 2-3 turns, then confuses t..."
6,dragon,physical,glaive-rush,120,100,No effect description available.
7,dragon,special,eternabeam,160,90,User foregoes its next turn to recharge.
8,electric,physical,bolt-strike,130,85,Has a 20% chance to paralyze the target.
9,electric,special,electro-shot,130,100,No effect description available.


The data above show the strongest moves for each type, and for each damage_type of that type. While it might be useful for choosing what move to put on which pokemon, it falls apart when you get into detail of these moves. Example:

In [23]:
run_query("""
    SELECT name, power, damage_class, type, effect
    FROM moves
    WHERE name = "explosion"
    OR name = "gigaton-hammer"
    OR name = "ice-burn"

    """)

,name,power,damage_class,type,effect
0,explosion,250,physical,normal,User faints.
1,gigaton-hammer,160,physical,steel,No effect description available.
2,ice-burn,140,special,ice,Requires a turn to charge before attacking. H...


As you can see, these moves have stronger power, but at the cost of heavy drawback of requiring a turn to charge the attack, or outright fainting the user after using the move. Gigaton Hammer seem like a a normal moves but have no effect description. Further researching find out that this move is
1. is a signature move of the pokemon Tinkaton, so only it can learn this move
2. cannot be use twice in a row
